# Note: my functions here aren't complete, only drafts.  For fully-functional functions, check out the hw3.py scripts.

In [21]:
%matplotlib inline

from __future__ import division, print_function

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.special import factorial
import time

#Typical plot parameters that make for pretty plots
mpl.rcParams['figure.figsize'] = (8,8)
mpl.rcParams['font.size'] = 15.0
mpl.rc('text', usetex='true') 

In [55]:
np.random.seed(int((time.time()%1)*1.0e8))

# GUI Forwarding when SSHing into Hyak

`ssh -X (hyak)` tells hyak you want X11 stuff sent to your computer.

`qsub -V -I` tells the scheduler that you want graphic stuff sent to you

# Rejection Method

https://en.wikipedia.org/wiki/Rejection_sampling

Rejection sampling is based on the observation that to sample a random variable one can sample uniformly from the region under the graph of its density function.

It's effectively doing a Monte Carlo integration to find the area under the curve.

Method to generate random variates that work for any distribution in any dimension!

You have some distribution P(x) and you want to generate x0 according to the distribution.

```
1) Draw box around distribution from [0,x_max] and [0,P_max] so box bounds distribution.

2) Sample uniform random number for both x, y from box bounds 

3) If a number doesn't fall under the distribution, reject it
    if you sample (x0,y0) and if y0 < P(x0), accept, otherwise try again
    
4) Try it N number of times and you'll get something below the curve

```

Notes:
- Method becomes inefficient if distribution is centrally peaked but a slow decline.
- Works better for distributions that have quickly declining tails.

Think of it this way:  Imagine you have a vary centrally peaked distribution with tails that die really quickly like a super narrow gaussian.  Any random (x0,y0) pair will be very unlikely to satisfy y0 < p(x0) in the tails since p(x0) is small, so not many x0 will be saved.  This recovered the distribution's behavior that not much stuff is out there.  If you hit the central peak, however, y0 < p(x0) will be likely to return true since p(x0) is large.  In this case, many x0 are returned recovering the behavior that the probability is high in that location.  The pileup of returned x0 is like the peak in the distribution (think histograms!).

In [1]:
def find_YMAX_poisson(lam):
    # Find the int that gives the max values
    # of the poisson distribution P(n), YMAX
    
    val = 0.0
    m = 0

    while True:
        tmp = poisson(m,lam)
        
        # If increasing, save value
        if tmp > val:
            val = tmp
            m_max = m
        else:
            break
    
        m = m + 1
    # end while
    
    return m_max, val
# end function
    
def find_XMAX_poisson(lam,m_max,eps=1.0e-8,MAX=50):
    # Given the index for the maximum of the poisson function,
    # find the index at which the Poisson distribution decays
    # to near 0, XMAX

    m = m_max
    count = 0

    # Start from peak and move rightward until distribution goes to 0
    while count < MAX:
        tmp = poisson(m,lam)
        
        # poisson is 0 enough
        if tmp < eps:
            return m
        
        m = m + 1
        count = count + 1
    
    return -1.0        
# end function
   
#####################
#
# Problem 1a
#   
#####################   
   
# Return n value drawn from the poisson
# 1 dimensional case for the poisson distribution
def nextPoisson(n,lam,MAX_ATTEMPTS=40):
      
    # Default inits    
    m_max, YMAX = find_YMAX_poisson(lam) 
    XMAX = find_XMAX_poisson(lam,m_max)    
    
    ret = []    
    
    for i in range(0,n):
        count = 0
        while(count < MAX_ATTEMPTS):
            x0 = np.random.uniform(low=0.0,high=XMAX)
            y0 = np.random.uniform(low=0.0,high=YMAX)
            
            # Increment count
            count += 1
            
            # Does it fall under the distribution
            if y0 < poisson(x0,lam):
                ret.append(x0)
                break
                
            # too many attempts!
            if count >= MAX_ATTEMPTS:
                print("Error: Too many attempts while calling rejection_sampling!")
                ret.append(0.0)
                break
        #end while
    
    return np.asarray(ret)
# end function
 
#####################
#
# Problem 2a
#   
#####################
           
# Returns n random deviates drawn from a Gaussian
# using the Box-Mueller Transform
# where args = (mu,sigma)
def nextGaussian(n,*args):

    # Extra args
    mu = args[0]
    sigma = args[1]
    
    # x1, x2 are uniform randoms from [0,1]
    # Draw 2 since BM transform returns 2 distributed according to normal distribution
    x1 = np.random.uniform(size=int(n/2))
    x2 = np.random.uniform(size=int(n/2))
    U1 = 1.0 - x1
    U2 = 1.0 - x2
    
    z0 = np.sqrt(-2.0 * np.log(U1)) * np.cos(2.0*np.pi * U2)
    z1 = np.sqrt(-2.0 * np.log(U1)) * np.sin(2.0*np.pi * U2)
    
    # Returns vector of normally distributed data!
    return np.concatenate([z0 * sigma + mu,z1 * sigma + mu])

# Sampling collisions/reflections in 3D spherical geometry
Quick aside:

P($\theta,\phi$)d$\Omega$

$\mu = 2x_1 - 1$

$\phi = 2 \pi x_2$

where x1, x2 are sampled uniformly from [0,1] and $\mu = \cos(\theta)$ and ranges from [-1,1].

# Markov Chain Monte Carlo

A Markov chain is a random process that makes a transition from one state to another based on a value of a random variable.  The choice of the next state depends only on the current state and not on any previous state (memoryless).

Example: Random walk
```
You have some grid where you have some probability to travel in one of 4 directions (1 = up, 2 = down, 3 = left, 4 = right).  You randomly choose which direction to go and the next state (location) only depends on the current one and the random number that decides the transition.
```

Not an Example: Self-avoiding random walk
```
Same as above, but you're not allowed to go back to where you have been.  Not a Markov chain BECAUSE it depends on previous states, that is, where you have already been.  Simulations of polymers employ this technique.
```

Example:
```
(state 1) --0.3--> (state 2)          (state 3)
        --->
          -0.7->         
             --->  (state x)
             
To see if you go from 1->2 or 1->x, sample uniformly from [0,1] to decide what transition to make.
```

In general, you have some starting state and potentially many different ending states.  Each transition from current state to some ending state has some probability where the total sums to 1, of course.  The choice of next state depends on some random variable.

To accomplish this, we can define a transition matrix. For a 3 state system, it would look like the following:
```
T = | P11 P12 P13 |
    | P21 P22 P23 |
    | P31 P32 P33 |

where P12 is the probability of transitions from 1->2.  In general P12 != P21 (!!).
```

Example: Random walk transition matrix
To accomplish this, we can define a transition matrix. For a 3 state system, it would look like the following:
```
T = | 1/4 0 |
    | 0  3/4|
In this example using the convention above for [[1,2],[3,4]] 2x2 transition matrix, you could only go up or right, but mostly likely right.
```
In this example, each time you run the simulation, you get a some number of steps.  The average number of steps goes as sqrt(N) for N transitions.  And this doesn't depend on the dimensions!


# Use of random walk to solve Laplace PDE at a single point.

https://en.wikipedia.org/wiki/Laplace%27s_equation

$$
\frac{\delta^2 u}{\delta x^2} + \frac{\delta^2 u}{\delta y^2} = 0
$$
with u given on some boundary.

You want to evaluate this at (x0,y0) and there's some boundary ub.  People have shown that
$$
u(x0,y0) = \frac{1}{N} \sum_i^N u_{b,i}
$$
where this is done where N is the number of walkers who random walk (markov chain style) to some boundary point.  So it's not like an average of ALL boundary points, but just the ones that the walkers explore.  So what happens is some random walker starts at (x0,y0) and uniformly random walks (think diffusion) through some grid until it hits some boundary point u_b,i.  Save that value for the sum later.

This technique is embarrassingly parallel (1 walker per core! Map reduce that!). You map the walkers to a bunch of machines, then reduce by doing the sums.  This technique is also great because it can handle an arbitrary boundary (as long as that boundary is well-known, of course).

# Multicore hyak (supercomputer) usage

How to run multicore jobs on 1 node using hyak.  This will be useful later on when we want to have a lot of walkers or something and we want to compute the mean and get a good (small) variance.

For batch jobs, write a PBS script (many examples on the hyak wiki).  Once you have a script ready to go, on a login node you call
```
qsub my_script.pbs
```
to run your stuff on an entire node, depending on what you ask for.

In the PBS script, include
```
cat mytasks | parallel
```

where mytaks is a file that has
```
mytask 1
mytask 2
...
mytask n
```

where each `mytask i` is a command to run, like python run_random_walk.py.

Parallel is smart and will run as many tasks at a time as there are cores.  To save output, make sure each task writes results to a certain place. So this stuff covers the map part, but you'll need to do the reduce step later.  If using the backfill queue, make sure to include check in your run.py or whatever to make sure that if it has already ran to not run it again.